## Langchain Integration for Nova

To aid in the development of applications with Nova models; Langchain support has been added for multimodal and agentic workflows.

## Setup

#### Installation

In [1]:
%pip install -q langchain langchain_community faiss_cpu pypdf --upgrade

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-aws 0.1.17 requires boto3<1.35.0,>=1.34.131, but you have boto3 1.35.79 which is incompatible.
langchain-aws 0.1.17 requires langchain-core<0.3,>=0.2.33, but you have langchain-core 0.3.24 which is incompatible.


**Note**: Micro can be used for text understanding use case and Lite and Pro for multimodal understanding use cases.

- Nova Micro: "us.amazon.nova-micro-v1:0"
- Nova Lite: "us.amazon.nova-lite-v1:0"
- Nova Pro: "us.amazon.nova-pro-v1:0"

## Model Invocation

#### Text Understanding

In [2]:
from langchain_aws import ChatBedrockConverse
from langchain_core.messages import HumanMessage

llm = ChatBedrockConverse(
    model_id="us.amazon.nova-lite-v1:0",
    temperature=0.7
)

messages = [
    ("system", "Provide three alternative song titles for a given user title"),
    ("user", "Teardrops on My Guitar"),
]

response = llm.invoke(messages)
print(f"Request ID: {response.id}")
response.pretty_print()


# Here we can pass the chat history to the model to ask follow up questions
multi_turn_messages = [
    *messages,
    response,
    HumanMessage(content="Select your favorite and tell me why"),
]

response = llm.invoke(multi_turn_messages)
print(f"\n\nRequest ID: {response.id}")
response.pretty_print()

C:\Users\Hoang Kha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\langchain_aws\chat_models\__init__.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_aws.chat_models.bedrock import BedrockChat, ChatBedrock
C:\Users\Hoang Kha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid c

SchemaError: Invalid Schema:
model.config.extra_fields_behavior
  Input should be 'allow', 'forbid' or 'ignore' [type=literal_error, input_value=<Extra.forbid: 'forbid'>, input_type=Extra]
    For further information visit https://errors.pydantic.dev/2.9/v/literal_error

#### Image Understanding

You are able to pass various media types to the model

In [ ]:
from IPython.display import Image

image_path = "media/sunset.png"
Image(filename=image_path)

In [ ]:
import base64

from langchain_aws import ChatBedrockConverse
from langchain_core.messages import HumanMessage

llm = ChatBedrockConverse(
    model="us.amazon.nova-lite-v1:0",
    temperature=0.7
)

with open(image_path, "rb") as image_file:
    binary_data = image_file.read()

message = HumanMessage(
    content=[
        {"image": {"format": "png", "source": {"bytes": binary_data}}},
        {"text": "Provide a summary of this photo"},
    ]
)

response = llm.invoke([message])
print(f"\n\nRequest ID: {response.id}")
response.pretty_print()

#### Video Understanding

In [ ]:
video_path = "media/cooking-quesadilla.mp4"

In [ ]:
from langchain_aws import ChatBedrockConverse

llm = ChatBedrockConverse(
    model="us.amazon.nova-lite-v1:0",
    temperature=0.7
)

with open(video_path, "rb") as video_file:
    binary_data = video_file.read()


message = HumanMessage(
    content=[
        {"video": {"format": "mp4", "source": {"bytes": binary_data}}},
        {"type": "text", "text": "Describe the following video"},
    ]
)

response = llm.invoke([message])
print(f"\n\nRequest ID: {response.id}")
response.pretty_print()

#### Streaming

Streaming is also supported

In [ ]:
from langchain_aws import ChatBedrockConverse
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

llm = ChatBedrockConverse(
    model="us.amazon.nova-lite-v1:0",
    temperature=0.7
)

chain = llm | StrOutputParser()

messages = [
    SystemMessage(content="You are an author with experience writing creative novels"),
    HumanMessage(
        content="Write an outlin for a novel about a wizard named Theodore graduating from college"
    ),
]

for chunk in chain.stream(messages):
    print(chunk, end="")

## Agent Workflows

The Nova model is capable of handling tool calling and agentic workflows.

#### Binding Tools

When using a model for tool calling you can take advantage of the bind tools method. This will pass a formatted tool config to the model. We recommend when taking advantage of tool calling or agentic workflows to use greedy decoding values. This means temperature=1, topP=1, topK=1

In [ ]:
from langchain_aws import ChatBedrockConverse
from langchain.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

tools = [multiply]

llm_with_tools = ChatBedrockConverse(
    model="us.amazon.nova-lite-v1:0",
    temperature=1,
    top_p=1,
    additional_model_request_fields={
        "inferenceConfig": {
            "topK": 1
        }
    },
).bind_tools(tools)

response = llm_with_tools.invoke([("user", "What is 8*8")])

print("[Model Response]\n")
print(response.content)

print("\n[Tool Calls]\n")
print(response.tool_calls)

#### Tool Calling Agents

For full workflows you can take advantage of custom parsers that will intercept outputs of the stream and allow you to invoke tools

In [ ]:
from langchain.agents import tool, AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrockConverse

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

tools = [multiply]

llm_with_tools = ChatBedrockConverse(
    model="us.amazon.nova-lite-v1:0",
    temperature=1,
    top_p=1,
    additional_model_request_fields={
        "inferenceConfig": {
            "topK": 1
        }
    },
).bind_tools(tools)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "What is 2*2?"})

## Structured Output

Structured output is a great way to force the model to return in a specific way. We use Greedy Decoding params here for more determinist results (Temperature = 1, Top P = 1, Top K = 1)

In [ ]:
from pydantic import BaseModel, Field
from langchain_aws import ChatBedrockConverse

class Joke(BaseModel):
    """A joke to respond to the user"""
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")

llm = ChatBedrockConverse(
    model="us.amazon.nova-lite-v1:0",
    temperature=1,
    top_p=1,
    additional_model_request_fields={
        "inferenceConfig": {
            "topK": 1
        }
    },
)

structured_llm = llm.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about cats")